# Generate CT slice video
Lung and abdomen windows with optional segmentations

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from concurrent.futures import ThreadPoolExecutor, wait
from pathlib import Path

import arrow
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk

from tqdm import tqdm
from skimage import measure

from video_utils import create_mp4
from plot_utils import plot_contours, plot_text, plot_slice_full, get_slice_range, plot_and_save

In [ ]:
dataset_folder = Path("/home/liushifeng/Desktop/AutoPET Lymphoma/dataset B_lymphoma")
seg_folder     = Path("/home/liushifeng/Desktop/AutoPET Lymphoma/SEG_Lymphoma_B_Rifki-selected")
frames_folder  = Path("../frames")
output_folder  = Path("../outputs")

ct_filename = "CT.nii.gz"
ct_paths = list(dataset_folder.glob(f"*/*/{ct_filename}"))

In [ ]:
dataset_name = dataset_folder.stem

dataset_B_lymphoma

In [ ]:
c = ct_paths[2]

In [ ]:
patient = c.parts[-3]
study = c.parts[-2]

In [ ]:
ct_frames_folder

In [ ]:
args = ct_array, seg_array, i, slice_range, "."
plot_and_save(args)

In [ ]:
for i, ct_path in enumerate(ct_paths):
    # if i < 9:
    #     continue
    print(ct_path.parts[-3:])
    patient = ct_path.parts[-3]
    study = ct_path.parts[-2]

    print(
        f"[{i:03d}/{len(ct_paths)}] "
        f"Processing: {patient} {study}"
    )
    
    scan_hash = ct_path.parent.parent.stem.split("_")[1]
    print(scan_hash)
    continue
    seg_path = list(seg_folder.glob(f"*{scan_hash}*"))[0]
    
    # load data
    ct_img = sitk.ReadImage(ct_path)
    seg_img = sitk.ReadImage(seg_path)
    seg_img = sitk.Resample(seg_img, ct_img, sitk.Transform(), sitk.sitkNearestNeighbor, 0.0, seg_img.GetPixelID())

    ct_array = sitk.GetArrayFromImage(ct_img)
    seg_array = sitk.GetArrayFromImage(seg_img)

    slice_range, n_slices = get_slice_range(seg_array)
    
    # create frames folder
    ct_frames_folder = frames_folder / dataset_slice_name
    ct_frames_folder.mkdir(exist_ok=True)

    break


    
    print(f"Generating {n_slices} frames")
    # for i in tqdm(range(*slice_range)):
    #     args = ct_array, seg_array, i, slice_range, ct_frames_folder
    #     plot_and_save(args)
    with ThreadPoolExecutor(max_workers=16) as e:
        futures = []
        for i in range(*slice_range):
            args = ct_array, seg_array, i, slice_range, ct_frames_folder
            futures.append(e.submit(plot_and_save, args))
        wait(futures)

    date_str = arrow.now().format("YYYY-MM-DD")
    output_path = output_folder / f"{dataset_slice_name}_video_{date_str}.mp4"

    print(f"Creating video: {output_path.name}")
    create_mp4(output_path, ct_frames_folder)
    print(f"Video saved as {output_path}")
    
    print("-"*30)

In [ ]:
# visualize a slice
i = 1360
fig = plot_slice_full(ct_array, seg_array, i, slice_range)
fig.savefig(f"test2.jpg", bbox_inches='tight', pad_inches=0, dpi=180);

## Window/visualize

In [ ]:
# Define window settings
lung_window = {'center': -600, 'width': 1500}
abdomen_window = {'center': 40, 'width': 400}

def apply_window(image, center, width):
    window_min = center - width // 2
    window_max = center + width // 2
    return np.clip(image, window_min, window_max)

In [ ]:
# Apply windowing
lung_ct = apply_window(ct_array, lung_window['center'], lung_window['width'])
abdomen_ct = apply_window(ct_array, abdomen_window['center'], abdomen_window['width'])

# Normalize for display
lung_ct = (lung_ct - lung_ct.min()) / (lung_ct.max() - lung_ct.min())
abdomen_ct = (abdomen_ct - abdomen_ct.min()) / (abdomen_ct.max() - abdomen_ct.min())

In [ ]:
# Select a slice for visualization (e.g., middle slice)
slice_idx = 1360

# Create overlay
# fig, ax = plt.subplots()
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

ax1.imshow(lung_ct[slice_idx], cmap='gray')
# ax1.imshow(seg_data[slice_idx], alpha=0.3, cmap='Reds')
ax1.set_title('Lung Window with Segmentation Overlay')

ax2.imshow(abdomen_ct[slice_idx], cmap='gray')
# ax2.imshow(seg_data[slice_idx], alpha=0.3, cmap='Reds')
ax2.set_title('Abdomen Window with Segmentation Overlay')

plt.show()